In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from MNIST datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders to iterate over data
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print("Training data size:", len(train_dataloader) * batch_size)
print("Test data size:", len(test_dataloader) * batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 491kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.60MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.88MB/s]

Training data size: 60032
Test data size: 10048
Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
device = torch.device(
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()  # Apple Silicon GPU
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [6]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, image_tensor):
        image_tensor = self.flatten(image_tensor)
        logits = self.linear_relu_stack(image_tensor)
        return logits

input_size = 28*28
hidden_size = 512
num_classes = 10

model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
learning_rate = 1e-3  # 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Let's define our training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Forward pass to compute prediction
        pred = model(X)
        # Compute prediction error using loss function
        loss = loss_fn(pred, y)

        # Backward pass
        optimizer.zero_grad()  # zero any previous gradient calculations
        loss.backward()  # calculate gradient
        optimizer.step()  # update model parameters

        if batch_num > 0 and batch_num % 100 == 0:
            loss, current = loss.item(), batch_num * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
# Let's run training
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.259589  [ 6400/60000]
loss: 0.201034  [12800/60000]
loss: 0.227928  [19200/60000]
loss: 0.157710  [25600/60000]
loss: 0.324994  [32000/60000]
loss: 0.138709  [38400/60000]
loss: 0.244276  [44800/60000]
loss: 0.195762  [51200/60000]
loss: 0.181191  [57600/60000]
Test Error: 
 Accuracy: 95.2%, Avg loss: 0.147417 

Epoch 2
-------------------------------
loss: 0.117077  [ 6400/60000]
loss: 0.097069  [12800/60000]
loss: 0.064232  [19200/60000]
loss: 0.026284  [25600/60000]
loss: 0.129301  [32000/60000]
loss: 0.051051  [38400/60000]
loss: 0.134731  [44800/60000]
loss: 0.089865  [51200/60000]
loss: 0.130065  [57600/60000]
Test Error: 
 Accuracy: 97.0%, Avg loss: 0.099923 

Epoch 3
-------------------------------
loss: 0.099417  [ 6400/60000]
loss: 0.049621  [12800/60000]
loss: 0.107644  [19200/60000]
loss: 0.080813  [25600/60000]
loss: 0.075835  [32000/60000]
loss: 0.051932  [38400/60000]
loss: 0.079470  [44800/60000]
loss: 0.078950  [51200/600

In [11]:
# Save our model parameters
torch.save(model.state_dict(), "ml_with_pytorch_model.pth")
print("Saved PyTorch Model State to ml_with_pytorch_model.pth")

# Load the saved model parameters into a new instance of the model
model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)
model.load_state_dict(torch.load("ml_with_pytorch_model.pth"))

# Inference using the new model instance
model.eval()
for i in range(10):
    x, y = test_data[i][0], test_data[i][1]

    x = x.to(device)
    pred = model(x)
    predicted, actual = pred[0].argmax(0).item(), y
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Saved PyTorch Model State to ml_with_pytorch_model.pth
Predicted: "7", Actual: "7"
Predicted: "2", Actual: "2"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "4"
Predicted: "1", Actual: "1"
Predicted: "4", Actual: "4"
Predicted: "9", Actual: "9"
Predicted: "5", Actual: "5"
Predicted: "9", Actual: "9"



**Building the model with 1 hidden layer**

In [12]:
class NeuralNetworkOneHiddenLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),  # input → hidden
            nn.ReLU(),                           # non-linearity
            nn.Linear(hidden_size, num_classes)  # hidden → output
        )

    def forward(self, x):
        x = self.flatten(x)           # [batch, 1, 28, 28] → [batch, 784]
        logits = self.linear_relu_stack(x)
        return logits

# Model setup
input_size = 28*28   # 784
hidden_size = 512    # still 512, but now only one hidden layer
num_classes = 10

model = NeuralNetworkOneHiddenLayer(input_size, hidden_size, num_classes).to(device)
print(model)

NeuralNetworkOneHiddenLayer(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
learning_rate = 1e-3  # 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Let's define our training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Forward pass to compute prediction
        pred = model(X)
        # Compute prediction error using loss function
        loss = loss_fn(pred, y)

        # Backward pass
        optimizer.zero_grad()  # zero any previous gradient calculations
        loss.backward()  # calculate gradient
        optimizer.step()  # update model parameters

        if batch_num > 0 and batch_num % 100 == 0:
            loss, current = loss.item(), batch_num * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [15]:
# Let's run training
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.282330  [ 6400/60000]
loss: 0.216166  [12800/60000]
loss: 0.255039  [19200/60000]
loss: 0.168707  [25600/60000]
loss: 0.294654  [32000/60000]
loss: 0.112995  [38400/60000]
loss: 0.297073  [44800/60000]
loss: 0.297350  [51200/60000]
loss: 0.209091  [57600/60000]
Test Error: 
 Accuracy: 95.5%, Avg loss: 0.143888 

Epoch 2
-------------------------------
loss: 0.131141  [ 6400/60000]
loss: 0.070487  [12800/60000]
loss: 0.114941  [19200/60000]
loss: 0.073759  [25600/60000]
loss: 0.167428  [32000/60000]
loss: 0.044939  [38400/60000]
loss: 0.191737  [44800/60000]
loss: 0.168557  [51200/60000]
loss: 0.146483  [57600/60000]
Test Error: 
 Accuracy: 96.6%, Avg loss: 0.103262 

Epoch 3
-------------------------------
loss: 0.085301  [ 6400/60000]
loss: 0.054605  [12800/60000]
loss: 0.045030  [19200/60000]
loss: 0.045366  [25600/60000]
loss: 0.102169  [32000/60000]
loss: 0.025323  [38400/60000]
loss: 0.118970  [44800/60000]
loss: 0.066869  [51200/600

**No hidden layer-no Relu; Logistic Regression**

In [16]:
class LogisticRegressionMNIST(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(input_size, num_classes)  # directly input → output

    def forward(self, x):
        x = self.flatten(x)       # [batch, 1, 28, 28] → [batch, 784]
        logits = self.linear(x)   # [batch, 784] → [batch, 10]
        return logits

# Model setup
input_size = 28*28   # 784 pixels
num_classes = 10     # digits 0–9

model = LogisticRegressionMNIST(input_size, num_classes).to(device)
print(model)

LogisticRegressionMNIST(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


In [17]:
learning_rate = 1e-3  # 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Let's define our training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch_num, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Forward pass to compute prediction
        pred = model(X)
        # Compute prediction error using loss function
        loss = loss_fn(pred, y)

        # Backward pass
        optimizer.zero_grad()  # zero any previous gradient calculations
        loss.backward()  # calculate gradient
        optimizer.step()  # update model parameters

        if batch_num > 0 and batch_num % 100 == 0:
            loss, current = loss.item(), batch_num * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
# Let's run training
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.752077  [ 6400/60000]
loss: 0.587166  [12800/60000]
loss: 0.521282  [19200/60000]
loss: 0.387217  [25600/60000]
loss: 0.430009  [32000/60000]
loss: 0.316032  [38400/60000]
loss: 0.500047  [44800/60000]
loss: 0.390481  [51200/60000]
loss: 0.436295  [57600/60000]
Test Error: 
 Accuracy: 90.9%, Avg loss: 0.335758 

Epoch 2
-------------------------------
loss: 0.331241  [ 6400/60000]
loss: 0.265067  [12800/60000]
loss: 0.389671  [19200/60000]
loss: 0.243207  [25600/60000]
loss: 0.370169  [32000/60000]
loss: 0.222480  [38400/60000]
loss: 0.431668  [44800/60000]
loss: 0.327607  [51200/60000]
loss: 0.425102  [57600/60000]
Test Error: 
 Accuracy: 91.8%, Avg loss: 0.295628 

Epoch 3
-------------------------------
loss: 0.299593  [ 6400/60000]
loss: 0.212137  [12800/60000]
loss: 0.345820  [19200/60000]
loss: 0.220704  [25600/60000]
loss: 0.353695  [32000/60000]
loss: 0.197119  [38400/60000]
loss: 0.407439  [44800/60000]
loss: 0.307763  [51200/600